<a href="https://colab.research.google.com/github/emma-rose22/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module4-logistic-regression/assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*
🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯🌯
---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [X] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [X] Begin with baselines for classification.
- [X] Use scikit-learn for logistic regression.
- [X] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [X] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [7]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [8]:
df.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
       'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
       'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
       'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
       'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
       'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
       'Great'],
      dtype='object')

###Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [0]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [0]:
train = df[(df['Date']<= '2016-12-31' )]
val = df[(df['Date'] >= '2017-01-01') &(df['Date'] <= '2017-12-31')]
test = df[(df['Date']>= '2018-01-01') & (df['Date'] <= '2020-12-31')]

In [33]:
train.shape, val.shape, test.shape

((298, 59), (85, 59), (37, 59))

In [35]:
df['Length'].dropna().mean()

20.038233215547702

### Begin with baselines for classification.

In [36]:
target = 'Great'
y_train = train[target]
y_train.value_counts(normalize=True)

False    0.590604
True     0.409396
Name: Great, dtype: float64

In [39]:
majority = y_train.mode()[0]
y_pred = [majority] * len(y_train)
len(y_pred)

298

In [40]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, y_pred)

#this is the score to beat 

0.5906040268456376

###Use scikit-learn for logistic regression.

####First attempt

In [0]:
import category_encoders as ce
from sklearn.linear_model import LogisticRegressionCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [0]:
features = ['Burrito', 'Cost', 'Hunger', 'Length', 'Tortilla', 'Temp', 'Uniformity', 'Salsa', 'Synergy', 'Wrap', 'Beef', 'Pico', 'Guac', 'Cheese',
           'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish', 'Avocado']

target = 'Great'

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [84]:
encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_encoded= encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)

X_train_encoded.shape, X_val_encoded.shape

((298, 43), (85, 43))

In [85]:
X_train_encoded.columns

Index(['Burrito_California', 'Burrito_Carnitas', 'Burrito_Asada',
       'Burrito_Other', 'Burrito_Surf & Turf', 'Cost', 'Hunger', 'Length',
       'Tortilla', 'Temp', 'Uniformity', 'Salsa', 'Synergy', 'Wrap', 'Beef_x',
       'Beef_nan', 'Beef_X', 'Pico_x', 'Pico_nan', 'Pico_X', 'Guac_x',
       'Guac_nan', 'Guac_X', 'Cheese_x', 'Cheese_nan', 'Cheese_X',
       'Sour cream_nan', 'Sour cream_x', 'Sour cream_X', 'Pork_nan', 'Pork_x',
       'Pork_X', 'Chicken_nan', 'Chicken_x', 'Chicken_X', 'Shrimp_nan',
       'Shrimp_x', 'Shrimp_X', 'Fish_nan', 'Fish_x', 'Fish_X', 'Avocado_nan',
       'Avocado_x'],
      dtype='object')

In [86]:
imputer = SimpleImputer()

X_train_imputed = imputer.fit_transform(X_train_encoded)
X_val_imputed = imputer.transform(X_val_encoded)

print(X_train_imputed.shape, X_val_imputed.shape)
X_train_imputed[:1]

(298, 43) (85, 43)


array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         6.49      ,  3.        , 19.82988571,  3.        ,  5.        ,
         4.        ,  4.        ,  4.        ,  4.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ]])

In [87]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

print(X_train_scaled.shape, X_val_scaled.shape)
X_train_imputed[:1]

(298, 43) (85, 43)


array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         6.49      ,  3.        , 19.82988571,  3.        ,  5.        ,
         4.        ,  4.        ,  4.        ,  4.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ]])

In [88]:
model = LogisticRegressionCV(cv=5, random_state=42)

model.fit(X_train_scaled, y_train)
print('Validation score1:', model.score(X_val_scaled, y_val))



Validation score1: 0.7529411764705882


In [92]:
%matplotlib inline
coefficients = pd.Series(model.coef_[0], X_train_encoded.columns)

coefficients.sort_values()

Burrito_Other         -0.157722
Sour cream_X          -0.142251
Cheese_X              -0.115610
Guac_nan              -0.099678
Beef_X                -0.099462
Pico_x                -0.092266
Chicken_x             -0.080235
Avocado_x             -0.079872
Shrimp_x              -0.074209
Pico_X                -0.068938
Fish_X                -0.067318
Chicken_X             -0.045918
Beef_x                -0.038933
Guac_X                -0.036128
Pork_x                -0.026895
Burrito_Asada          0.000815
Pork_X                 0.011514
Fish_x                 0.014573
Pork_nan               0.015752
Sour cream_nan         0.018682
Cheese_nan             0.024077
Fish_nan               0.031466
Length                 0.036077
Shrimp_X               0.037849
Cheese_x               0.044174
Burrito_Surf & Turf    0.048754
Burrito_Carnitas       0.050132
Shrimp_nan             0.053685
Wrap                   0.066513
Sour cream_x           0.070441
Avocado_nan            0.079872
Chicken_

####Second attempt

(beat 75.3)

only removed the features with least contribution

In [93]:
df.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
       'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
       'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
       'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
       'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
       'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
       'Great'],
      dtype='object')

In [0]:
features2 = ['Burrito', 'Cost', 'Hunger', 'Tortilla', 'Temp', 'Uniformity', 'Salsa', 'Synergy', 'Beef', 'Guac', 'Cheese',
           'Sour cream', 'Pork']

target = 'Great'

X_train2 = train[features2]
y_train2 = train[target]
X_val2 = val[features2]
y_val2 = val[target]

In [98]:
encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_encoded2= encoder.fit_transform(X_train2)
X_val_encoded2 = encoder.transform(X_val2)

X_train_encoded2.shape, X_val_encoded2.shape

((298, 27), (85, 27))

In [99]:
imputer = SimpleImputer()

X_train_imputed2 = imputer.fit_transform(X_train_encoded2)
X_val_imputed2 = imputer.transform(X_val_encoded2)

print(X_train_imputed2.shape, X_val_imputed2.shape)
X_train_imputed2[:1]

(298, 27) (85, 27)


array([[1.  , 0.  , 0.  , 0.  , 0.  , 6.49, 3.  , 3.  , 5.  , 4.  , 4.  ,
        4.  , 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  ,
        0.  , 0.  , 1.  , 0.  , 0.  ]])

In [100]:
scaler = StandardScaler()

X_train_scaled2 = scaler.fit_transform(X_train_imputed2)
X_val_scaled2 = scaler.transform(X_val_imputed2)

print(X_train_scaled2.shape, X_val_scaled2.shape)
X_train_imputed2[:1]

(298, 27) (85, 27)


array([[1.  , 0.  , 0.  , 0.  , 0.  , 6.49, 3.  , 3.  , 5.  , 4.  , 4.  ,
        4.  , 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  ,
        0.  , 0.  , 1.  , 0.  , 0.  ]])

In [101]:
model2 = LogisticRegressionCV(cv=5, random_state=42)

model2.fit(X_train_scaled2, y_train2)
print('Validation score2:', model2.score(X_val_scaled2, y_val2))


Validation score2: 0.7647058823529411


###Third attempt

(beat 76.5)

adding in features not origionally used/removed in second attempt

In [0]:
features3 = ['Burrito', 'Cost', 'Hunger', 'Tortilla', 'Temp', 'Uniformity', 'Salsa', 'Synergy', 'Beef', 'Guac', 'Cheese',
           'Sour cream', 'Pork', 'Circum', 'Volume', 'Meat', 'Fillings', 'Meat:filling', 'Sauce', 'Rice', 'Beans' ]

target = 'Great'

X_train3 = train[features3]
y_train3 = train[target]
X_val3 = val[features3]
y_val3 = val[target]

In [165]:
len(features3)

21

In [166]:
encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_encoded3= encoder.fit_transform(X_train3)
X_val_encoded3 = encoder.transform(X_val3)

X_train_encoded3.shape, X_val_encoded3.shape

((298, 41), (85, 41))

In [167]:
imputer = SimpleImputer()

X_train_imputed3 = imputer.fit_transform(X_train_encoded3)
X_val_imputed3 = imputer.transform(X_val_encoded3)

print(X_train_imputed3.shape, X_val_imputed3.shape)
X_train_imputed3[:1]

(298, 41) (85, 41)


array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         6.49      ,  3.        ,  3.        ,  5.        ,  4.        ,
         4.        ,  4.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        , 22.04224138,  0.77091954,  3.        ,
         3.5       ,  4.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ]])

In [168]:
scaler = StandardScaler()

X_train_scaled3 = scaler.fit_transform(X_train_imputed3)
X_val_scaled3 = scaler.transform(X_val_imputed3)

print(X_train_scaled3.shape, X_val_scaled3.shape)
X_train_imputed3[:1]

(298, 41) (85, 41)


array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         6.49      ,  3.        ,  3.        ,  5.        ,  4.        ,
         4.        ,  4.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        , 22.04224138,  0.77091954,  3.        ,
         3.5       ,  4.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ]])

In [169]:
model3 = LogisticRegressionCV(cv=5, random_state=42)

model3.fit(X_train_scaled3, y_train3)
print('Validation score3:', model3.score(X_val_scaled3, y_val3))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Validation score3: 0.8705882352941177


In [170]:
model.coef_[0], X_train_encoded3.columns

(array([ 1.07888155e-01,  5.01321145e-02,  8.14894874e-04, -1.57721961e-01,
         4.87538235e-02,  3.19422561e-01,  1.03145639e-01,  3.60768355e-02,
         3.68726250e-01,  2.95953413e-01,  2.80340374e-01,  3.34200185e-01,
         9.54265612e-01,  6.65131323e-02, -3.89330035e-02,  1.05830659e-01,
        -9.94623558e-02, -9.22655753e-02,  1.30165262e-01, -6.89378126e-02,
         1.30512415e-01, -9.96781739e-02, -3.61283527e-02,  4.41737706e-02,
         2.40768659e-02, -1.15609958e-01,  1.86819671e-02,  7.04406232e-02,
        -1.42251483e-01,  1.57518092e-02, -2.68949939e-02,  1.15139859e-02,
         8.89564457e-02, -8.02349779e-02, -4.59175097e-02,  5.36848243e-02,
        -7.42085393e-02,  3.78485660e-02,  3.14664361e-02,  1.45732961e-02,
        -6.73184308e-02,  7.98721092e-02, -7.98721092e-02]),
 Index(['Burrito_California', 'Burrito_Carnitas', 'Burrito_Asada',
        'Burrito_Other', 'Burrito_Surf & Turf', 'Cost', 'Hunger', 'Tortilla',
        'Temp', 'Uniformity', 'Sal

###Get your model's validation accuracy. (Multiple times if you try multiple iterations.)

In [61]:
print('Validation score 1:', model.score(X_val_scaled, y_val))


Validation score 1: 0.7529411764705882


In [102]:
print('Validation score2:', model2.score(X_val_scaled2, y_val2))

Validation score2: 0.7647058823529411


In [171]:
print('Validation score3:', model3.score(X_val_scaled3, y_val3))

Validation score3: 0.8705882352941177


###Get your model's test accuracy. (One time, at the end.)

In [181]:
X_test = test[features3]
X_test_encoded = encoder.transform(X_test)
X_test_imputed = imputer.transform(X_test_encoded)
X_test_scaled = scaler.transform(X_test_imputed)
model3.predict(X_test_scaled)


array([ True, False,  True, False, False,  True,  True, False,  True,
        True, False, False, False,  True,  True,  True, False, False,
       False, False,  True, False,  True,  True, False, False,  True,
        True,  True, False,  True,  True, False,  True, False,  True,
        True])

In [0]:
y_test = test[target]

In [186]:
(y_test==y_pred).sum()/len(y_test)

0.8108108108108109

In [187]:
accuracy_score(y_test, y_pred)

0.8108108108108109